In [14]:
import xarray as xr
import numpy as np
import dask
import matplotlib.pyplot as plt
from utils import geo
import pandas as pd
import regionmask
from dask.diagnostics import ProgressBar
import glob
import os

In [6]:
dask.config.set(**{'array.slicing.split_large_chunks': False})

In [7]:
dataset = 'globcolour'
frequency = 'day'
variable = 'CHL1'
yearstart = 1999
yearend = 2022
monthorder = 'JULtoJUN'
averaging = 'wmean-latlon'

In [8]:
if dataset == 'globcolour':
    rootdir = '/projects/SOCCOM/datasets/GlobColour/merged/'
    mode = 'AV'
    
def preprocess(ds):
    time = pd.to_datetime(ds.attrs['start_time'])
    if 'CHL1_error' in ds.data_vars:
        ds = ds.drop(['CHL1_error'])
    return ds.sel({'lat':slice(-30,-65)}).expand_dims({'time':[time]})

print('Opening.')
paths = glob.glob(rootdir+frequency+'/*/*/*/*'+mode+'*'+variable+'*.nc')
ds = xr.open_mfdataset(paths,preprocess=preprocess,use_cftime=False)
ds['time'] = pd.to_datetime(ds['time'].values)
# Get area
ds,xgrid = geo.get_xgcm_horizontal(ds,axes_dims_dict={'X':'lon','Y':'lat'},periodic='X')
ds['area'] = ds['dxC']*ds['dyC']
# Rename CHL1_mean to CHL1
ds = ds.rename({'CHL1_mean':variable})

Opening.


In [9]:
ds = ds.chunk({'time':5})
ds

<xarray.Dataset>
Dimensions:     (time: 9230, lat: 840, lon: 8640, lon_left: 8640, lat_left: 840)
Coordinates:
  * time        (time) datetime64[ns] 1997-09-04T16:26:34 ... 2023-01-04T00:0...
  * lat         (lat) float32 -30.02 -30.06 -30.1 -30.15 ... -64.9 -64.94 -64.98
  * lon         (lon) float32 -180.0 -179.9 -179.9 -179.9 ... 179.9 179.9 180.0
  * lon_left    (lon_left) float32 -180.0 -180.0 -179.9 ... 179.9 179.9 180.0
  * lat_left    (lat_left) float32 -30.0 -30.04 -30.08 ... -64.88 -64.92 -64.96
Data variables:
    CHL1        (time, lat, lon) float32 dask.array<chunksize=(5, 840, 8640), meta=np.ndarray>
    CHL1_flags  (time, lat, lon) float32 dask.array<chunksize=(5, 840, 8640), meta=np.ndarray>
    dxG         (lon_left, lat) float32 dask.array<chunksize=(8640, 840), meta=np.ndarray>
    dyG         (lon, lat_left) float32 dask.array<chunksize=(8640, 840), meta=np.ndarray>
    dxC         (lon, lat) float32 dask.array<chunksize=(8640, 840), meta=np.ndarray>
    dyC         (lon, lat) float32 dask.array<chunksize=(8640, 840), meta=np.ndarray>
    rC          (lon, lat) float32 dask.array<chunksize=(8640, 840), meta=np.ndarray>
    area        (lon, lat) float32 dask.array<chunksize=(8640, 840), meta=np.ndarray>
Attributes: (12/53)
    Conventions:                CF-1.4
    title:                      GlobColour daily SeaWiFS product
    product_name:               L3m_19970904__GLOB_4_AV-SWF_CHL1_DAY_00.nc
    product_type:               day
    product_version:            2018.1
    product_level:              3
    ...                         ...
    references:                 http://www.globcolour.info
    contact:                    service@globcolour.info
    copyright:                  Copyright ACRI-ST - GlobColour. GlobColour ha...
    history:                    20180425T205446Z: globcolour_l3_reproject.sh ...
    input_files:                S1997247162631.L2_GAC_OC.nc,S1997247162634.L2...
    input_files_reprocessings:  2018.0,2018.0

In [10]:
# Apply a landmask
mask = regionmask.defined_regions.natural_earth.land_110.mask(ds['lon'], ds['lat'])
mask = mask.where(np.isfinite(mask),1)
mask = mask.where(mask==1,np.nan)
# mask.plot()
ds[variable] = ds[variable].fillna(0)*mask

/home/graemem/.conda/envs/core/lib/python3.10/site-packages/regionmask/defined_regions/_natural_earth.py:471: FutureWarning: ``regionmask.defined_regions.natural_earth`` is deprecated. Please use ``regionmask.defined_regions.natural_earth_v4_1_0`` or ``regionmask.defined_regions.natural_earth_v5_0_0`` instead.
  warnings.warn(


In [11]:
# Extract spatial average for each year
years = np.arange(yearstart,yearend+1)
days = np.arange(0,365)
# Ordering such that years run from July to June
ds_year = xr.DataArray(dims=['day','year'],coords={'year':years,'day':days},name=variable)
for i,year in enumerate(years):
    print(year)
    if monthorder=='JULtoJUN':
        start = str(year-1)+'-07-01'
        end = str(year)+'-06-30'
        nleapday = 243 # for removing leap day (243 days after July 1)
    elif monthorder=='JANtoDEC':
        start = str(year-1)+'-01-01'
        end = str(year)+'-12-31'
        nleapday = 59 # for removing leap day (59 days after Jan 1)
    
    # Weighted spatial mean
    x = (ds[variable].sel({'time':slice(start,end)})).weighted(ds['area'].fillna(0)).mean(['lat','lon'])
    
    # Interpolate to daily
    # (day frquency data has some missing days, so interpolate that too)
    alltimes = pd.date_range(start,end)
    xi = x.interp({'time':alltimes},kwargs={'fill_value':np.nan})
        
    # Remove leap-year day for simplicity
    if len(xi)==366:
        xi = xi[np.arange(len(xi))!=nleapday]
        
    # Put into dataset
    with ProgressBar():
        xi = xi.compute()
    ds_year.loc[{'year':year}]=xi.values

1999
[########################################] | 100% Completed | 105.09 s
2000
[########################################] | 100% Completed | 108.76 s
2001
[########################################] | 100% Completed | 109.20 s
2002
[########################################] | 100% Completed | 112.09 s
2003
[########################################] | 100% Completed | 147.61 s
2004
[########################################] | 100% Completed | 148.28 s
2005
[########################################] | 100% Completed | 145.51 s
2006
[########################################] | 100% Completed | 147.24 s
2007
[########################################] | 100% Completed | 146.44 s
2008
[########################################] | 100% Completed | 145.54 s
2009
[########################################] | 100% Completed | 147.75 s
2010
[########################################] | 100% Completed | 156.54 s
2011
[########################################] | 100% Completed | 141.60 s
2012
[######

In [12]:
outdir = '../../data/'
yearstr = str(yearstart)+'-'+str(yearend)
outpath = outdir+'.'.join([dataset,frequency,averaging,monthorder,yearstr,'nc'])

In [13]:
print('Saving to '+outpath)
if os.path.isfile(outpath):
    os.remove(outpath)
ds_year.to_netcdf(outpath)

Saving to ../../data/globcolour.day.wmean-latlon.JULtoJUN.1999-2022.nc


### By basin

In [61]:
atlind = 20
indwpac = 170
wpacepac = -130
epacatl = -70

cond_atlantic = (ds['lon']>=epacatl) & (ds['lon']<atlind)
cond_indian = (ds['lon']>=atlind) & (ds['lon']<indwpac)
cond_wpacific = (ds['lon']>=indwpac) | (ds['lon']<wpacepac)
cond_epacific = (ds['lon']>=wpacepac) & (ds['lon']<epacatl)

atlantic = ds[variable].where(cond_atlantic,drop=True)
atlantic_area = ds['area'].where(cond_atlantic,drop=True)
indian = ds[variable].where(cond_indian,drop=True)
indian_area = ds['area'].where(cond_indian,drop=True)
wpacific = ds[variable].where(cond_wpacific,drop=True)
wpacific_area = ds['area'].where(cond_wpacific,drop=True)
epacific = ds[variable].where(cond_epacific,drop=True)
epacific_area = ds['area'].where(cond_epacific,drop=True)

In [62]:
# Extract spatial average for each year
atlantic_year = xr.DataArray(dims=['day','year'],coords={'year':years,'day':days},name=variable)
indian_year = xr.DataArray(dims=['day','year'],coords={'year':years,'day':days},name=variable)
wpacific_year = xr.DataArray(dims=['day','year'],coords={'year':years,'day':days},name=variable)
epacific_year = xr.DataArray(dims=['day','year'],coords={'year':years,'day':days},name=variable)
for i,year in enumerate(years):
    print(year)
    start = str(year-1)+'-07-01'
    end = str(year)+'-06-30'
    alltimes = pd.date_range(start,end)
    
    # Atlantic
    print('atlantic')
    x = (atlantic.sel({'time':slice(start,end)})).weighted(atlantic_area.fillna(0)).mean(['lat','lon'])
    x = x.interp({'time':alltimes})
    if len(x)==366:
        # Just take out the leap year day
        # (Always 243 days after July 1)
        x = x[np.arange(len(x))!=243]
    # Put into dataset
    with ProgressBar():
        atlantic_year.loc[{'year':year}]=x.values
        
    # Indian
    print('indian')
    x = (indian.sel({'time':slice(start,end)})).weighted(indian_area.fillna(0)).mean(['lat','lon'])
    x = x.interp({'time':alltimes})
    if len(x)==366:
        # Just take out the leap year day
        # (Always 243 days after July 1)
        x = x[np.arange(len(x))!=243]
    # Put into dataset
    with ProgressBar():
        indian_year.loc[{'year':year}]=x.values
        
    # West Pacific
    print('wpacific')
    x = (wpacific.sel({'time':slice(start,end)})).weighted(wpacific_area.fillna(0)).mean(['lat','lon'])
    x = x.interp({'time':alltimes})
    if len(x)==366:
        # Just take out the leap year day
        # (Always 243 days after July 1)
        x = x[np.arange(len(x))!=243]
    # Put into dataset
    with ProgressBar():
        wpacific_year.loc[{'year':year}]=x.values
        
    # East Pacific
    print('epacific')
    x = (epacific.sel({'time':slice(start,end)})).weighted(epacific_area.fillna(0)).mean(['lat','lon'])
    x = x.interp({'time':alltimes})
    if len(x)==366:
        # Just take out the leap year day
        # (Always 243 days after July 1)
        x = x[np.arange(len(x))!=243]
    # Put into dataset
    with ProgressBar():
        epacific_year.loc[{'year':year}]=x.values

1998
atlantic
[########################################] | 100% Completed | 13.92 s
indian
[########################################] | 100% Completed | 12.38 s
wpacific
[########################################] | 100% Completed | 9.95 ss
epacific
[########################################] | 100% Completed | 9.08 ss
1999
atlantic
[########################################] | 100% Completed | 20.04 s
indian
[########################################] | 100% Completed | 12.83 s
wpacific
[########################################] | 100% Completed | 10.34 s
epacific
[########################################] | 100% Completed | 10.53 s
2000
atlantic
[########################################] | 100% Completed | 28.60 s
indian
[########################################] | 100% Completed | 14.46 s
wpacific
[########################################] | 100% Completed | 12.11 s
epacific
[########################################] | 100% Completed | 10.24 s
2001
atlantic
[############################

In [63]:
for basin in ['atlantic','indian','wpacific','epacific']:
    outpath = outdir+'.'.join([dataset,frequency,averaging,monthorder,yearstr,basin,'nc'])
    if basin=='atlantic':
        atlantic_year.to_netcdf(outpath)
    elif basin=='indian':
        indian_year.to_netcdf(outpath)
    elif basin=='wpacific':
        wpacific_year.to_netcdf(outpath)
    elif basin=='epacific':
        epacific_year.to_netcdf(outpath)